# Patronus `Lynx` Hallucination Detection

In AI systems, the term **Hallucination** is used to describe cases where an AI model produces responses that are broadly defined as not factual. For example, if you ask an LLM, `What is the atomic number of oxygen?` and it returns `15`, that would be considered a **Hallucination**, as the correct atomic number of oxygen is `8`.


**Hallucinations** are of course worrisome for all cases of AI systems, but they are especially important for systems that utilize Retreival-Augmented Generation (RAG). RAG is generally predicated upon the promise of equipping LLMs with knowledge about your particular context, and it is thus especially important to make sure this is indeed being achieved.

A recent [paper from researchers at Yale University](https://arxiv.org/pdf/2504.17004) highlights that completely avoiding **Hallucinations** in LLMs is extremely difficult, if not impossible. It is very unlikely that the next generation of LLMs will magically solve this problem with increased compute, new neural network architectures, or more training data.

Fortunately, we have several techniques to combat **Hallucinations** that are getting stronger and stronger, especially in the context of RAG. In this notebook, we will firstly present the `Lynx` model from Patronus AI. As explained further in their [technical report](https://arxiv.org/abs/2407.08488), `Lynx` is a state-of-the-art model for **Hallucination** detection.

From the Weaviate side of the house, we have kicked off our efforts to fight **Hallucinations** with the Weaviate Query Agent by instructing the model to cite its sources with a structured output model and return a `sources` output to the user in the final response.

This notebook will illustrate how to evaluate if the Weaviate Query Agent is **hallucinating** by connecting the returned `sources` with `Lynx` and Patronus AI's observability platform!

> This notebook is using patronus `0.1.3`

Authored by Connor Shorten and Josh Goldstein. May 6th, 2025.

# ![Patronus AI](./images/patronus-logo.png)

### Illustration of Patronus AI's `Lynx` model

This Python code uses the Patronus library to evaluate AI responses for hallucinations. It sets up a hallucination detector that checks if an AI's answer about `the largest animal in the world` (claiming it's "the giant sandworm") aligns with the provided factual context stating **blue whales** hold this title, rather than the fictional sandworms from Dune. The code demonstrates automated detection of when AI systems provide fictional information instead of factual answers.

In [2]:
import os
import patronus
from patronus.evals import RemoteEvaluator

patronus.init(
    os.getenv("PATRONUS_API_KEY")
)

patronus_evaluator = RemoteEvaluator("lynx", "patronus:hallucination")
# See other built-in evaluators here - https://docs.patronus.ai/docs/evaluation_api/reference_guide

result = patronus_evaluator.evaluate(
    task_input="What is the largest animal in the world?",
    task_context=["The blue whale is the largest known animal on the planet.","In Dune by Frank Herbert, Sandworms are the largest animals - beware if you like spice!"],
    task_output="The giant sandworm.",
    gold_answer=""
)

print(result)

score=0.01 pass_=False text_output=None metadata={'positions': [[0, 19]], 'extra': None, 'confidence_interval': None} explanation='\'The context mentions that the blue whale is the largest known animal on the planet.\', \'The context also mentions that in the book Dune, Sandworms are the largest animals, but this is in a fictional context.\', "The answer \'The giant sandworm\' is not faithful to the context because it incorrectly identifies a fictional entity as the largest animal in the world, whereas the context clearly states that the blue whale is the largest known animal."' tags={} dataset_id=None dataset_sample_id=None evaluation_duration=datetime.timedelta(microseconds=940000) explanation_duration=datetime.timedelta(0)


### Import Weaviate Blogs to Weaviate

We will now test Hallucination Detection by answering questions about information contained in Weaviate's Blog Posts.

In [3]:
import os
import weaviate

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
)

In [5]:
import os
import tiktoken
import time

import weaviate
import weaviate.collections.classes.config as wvcc
from dotenv import load_dotenv
from weaviate.classes.init import AdditionalConfig, Timeout

load_dotenv()

local_blogs = []

# The blogs dataset can be found in recipes within `integrations/llm-agent-frameworks/data`
# You can also get it from `github.com/weaviate-io/blog``
main_folder_path = "./blog/"

for i, folder_name in enumerate(os.listdir(main_folder_path)):
    subfolder_path = os.path.join(main_folder_path, folder_name)
    if os.path.isdir(subfolder_path):
        index_file_path = os.path.join(subfolder_path, "index.mdx")
        if os.path.isfile(index_file_path):
            with open(index_file_path, "r", encoding="utf-8") as file:
                content = file.read()
                local_blogs.append(
                    {
                        "content": content,
                    }
                )

if weaviate_client.collections.exists("Blogs"):
    weaviate_client.collections.delete("Blogs")
blogs = weaviate_client.collections.create(
    name="Blogs",
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_weaviate(),
    properties=[
        wvcc.Property(name="content", data_type=wvcc.DataType.TEXT),
    ],
)

def chunk_text(text, max_tokens=300):
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(text)
    chunks = []
    
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = enc.decode(chunk_tokens)
        chunks.append(chunk_text)
    
    return chunks

chunked_blogs = []
for blog in local_blogs:
    chunks = chunk_text(blog["content"])
    for chunk in chunks:
        chunked_blogs.append({
            "content": chunk
        })

start_time = time.time()
with weaviate_client.batch.dynamic() as batch:
    for blog_chunk in chunked_blogs:
        batch.add_object(
            collection="Blogs",
            properties={
                "content": blog_chunk["content"],
            }
        )
end_time = time.time()
upload_time = end_time - start_time

print(f"Successfully imported {len(chunked_blogs)} blog chunks into Weaviate.")
print(f"Upload time: {upload_time:.2f} seconds")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/weaviate/collections/classes/config.py:1950: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:


Successfully imported 1463 blog chunks into Weaviate.
Upload time: 9.45 seconds


### Weaviate Query Agent

The Weaviate Query Agent returns meta information about its execution in addition to the `final_answer`, such as `sources`.

In [25]:
from weaviate.agents.query import QueryAgent
from weaviate.agents.utils import print_query_agent_response

qa = QueryAgent(
    client=weaviate_client, collections=["Blogs"]
)

response = qa.run("How does HNSW work?")
print_query_agent_response(response)

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ How does HNSW work?                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ HNSW, or Hierarchical Navigable Small World, is an algorithm designed for efficient approximate nearest         │
│ neighbor (ANN) search in high-dimensional spaces. HNSW builds a hierarchical, multi-layered proximity graph to  │
│ organize data vectors, enabling rapid navigation and search.                                                    │
│                                                                                                                 │
│ Here's how HNSW works:                                                                                          │
│ - **Hierarchy of Layers:** The index consists of multiple graph layers. Upper layers contain nodes with         │
│ long-range connections, while lower layers hold more densely connected nodes with shorter-range (local) edges.  │
│ - **Search Process:** When searching for the nearest neighbors of a query vector, the search starts at the top  │
│ layer, where the graph allows for large jumps across the data using long-range connections. As the algorithm    │
│ descends layer by layer, it focuses more locally, narrowing the search to nodes more likely to be close to the  │
│ query in high-dimensional space.                                                                                │
│ - **Traversal:** This hierarchical structure enables the search to quickly zero in on relevant regions of the   │
│ dataset, significantly reducing the number of comparisons required compared to a flat (single-layer) or         │
│ brute-force search.                                                                                             │
│ - **Tradeoff:** HNSW delivers very high recall (accuracy of finding true neighbors) with much improved search   │
│ speed and low latency, by returning approximate (but very close) results rather than exact nearest neighbors.   │
│                                                                                                                 │
│ Overall, HNSW is well-suited for large-scale vector search scenarios, such as those found in modern vector      │
│ databases, where performance and scalability are critical.                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['how Hierarchical Navigable Small World (HNSW) algorithm works'],                                  │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Blogs'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='01a7eda0-729a-466a-bdfe-3ad8e18c36f6' collection='Blogs'                                          │
│  - object_id='ba0ef4c5-a4f0-45c6-8318-fd91e739ae64' collection='Blogs'                                          │
│  - object_id='f5c5d781-75eb-4728-a037-6ee2406e4c26' collection='Blogs'                                          │
│  - object_id='58d43298-0672-42f5-a0ed-2cccf49c4d16' collection='Blogs'                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 8859 │
│ Output Tokens: │ 411  │
│ Total Tokens:  │ 9270 │
└────────────────┴──────┘

Total Time Taken: 10.77s

### `sources` from the Query Agent

The response from the Weaviate Query Agent contains a `sources` field that can be used to help users understand what influenced the final answer.

In [26]:
response.sources

[Source(object_id='01a7eda0-729a-466a-bdfe-3ad8e18c36f6', collection='Blogs'),
 Source(object_id='ba0ef4c5-a4f0-45c6-8318-fd91e739ae64', collection='Blogs'),
 Source(object_id='f5c5d781-75eb-4728-a037-6ee2406e4c26', collection='Blogs'),
 Source(object_id='58d43298-0672-42f5-a0ed-2cccf49c4d16', collection='Blogs')]

### Helper function to pass `sources` --> `Lynx` Hallucination Detection

In [27]:
from weaviate.classes.query import Filter

blogs_collection = weaviate_client.collections.get("Blogs")

def format_source_content(sources, collection):
    """
    Formats the content from source objects into a clean, numbered format.
    
    Args:
        sources: List of source objects with object_id attribute
        collection: Weaviate collection to query
        
    Returns:
        str: Formatted string of all search results
    """
    source_uuids = [source.object_id for source in sources]
    
    objects = collection.query.fetch_objects_by_ids(
        source_uuids
    )
    
    # Format the search results in a clean, numbered format
    search_results = []
    for i, o in enumerate(objects.objects, 1):
        content = o.properties.get('content', '')
        search_results.append(f"Search Result #{i}\n\n{content}\n")
    
    # Join all results into a single string
    formatted_results = "\n".join(search_results)
    return formatted_results

# Use the function with the response sources
formatted_results = format_source_content(response.sources, blogs_collection)
print(formatted_results)

Search Result #1

first step**, you can see that the entry point for the search is in the center, and then the long-range connections allow jumping to the edges. This means that when a query comes, it will quickly move in the appropriate direction.<br/>
The **second**, **third**, and **final steps** highlight the nodes reachable within **three**, **six**, and **nine** hops from the entry node.

HNSW, on the other hand, implements the same idea a bit differently. Instead of having all information together on a flat graph, it has a hierarchical representation distributed across multiple layers. The top layers only contain long-range connections, and as you dive deeper into the layers, your query is routed to the appropriate region where you can look more locally for your answer. So your search starts making only big jumps across the top layers until it finally looks for the closest points locally in the bottom layers.

## Performance comparison
So, how do they perform? Let's take a look 

### Evaluate Query Agent Hallucination with Patronus `Lynx`

In [28]:
result = patronus_evaluator.evaluate(
    task_input="How does HNSW work?",
    task_context=[formatted_results],
    task_output=response.final_answer,
    gold_answer=""
)

print(result.score)
print(result.pass_)
print(result.explanation)

1.0
True
- The QUESTION asks about how HNSW works.
- The CONTEXT provides detailed information about HNSW, including its purpose, structure, and search process.
- The ANSWER summarizes the key points from the CONTEXT, explaining that HNSW is an algorithm for efficient approximate nearest neighbor (ANN) search in high-dimensional spaces.
- The ANSWER describes the hierarchical structure of HNSW, mentioning the multiple layers and the transition from long-range connections in upper layers to shorter-range connections in lower layers.
- The ANSWER explains the search process, starting at the top layer with long-range connections and gradually focusing more locally as the search descends through the layers.
- The ANSWER highlights the benefits of HNSW, including high recall, improved search speed, and low latency, by returning approximate results rather than exact nearest neighbors.
- The ANSWER concludes that HNSW is well-suited for large-scale vector search scenarios, aligning with the C

# ![Patronus AI](./images/patronus-gui.png)

### Run more tests

In [32]:
def run_and_evaluate_query(query):
    # Run the query
    response = qa.run(query)
    
    print(response.final_answer)
    print("Evaluating with Lynx...")
    
    # Format the source content
    formatted_results = format_source_content(response.sources, blogs_collection)
    
    # Evaluate with Patronus
    result = patronus_evaluator.evaluate(
        task_input=query,
        task_context=[formatted_results],
        task_output=response.final_answer,
        gold_answer=""
    )
    
    print(result.score)
    print(result.pass_)
    print(result.explanation)

query = "What does ACORN stand for?"
run_and_evaluate_query(query)

ACORN stands for ANN Constraint-Optimized Retrieval Network. It is a filter strategy used to speed up filtered vector search, especially in systems like Weaviate. ACORN enhances traditional HNSW (Hierarchical Navigable Small World) algorithms by maintaining high connectivity in the graph through techniques like two-hop neighborhood expansion and adaptive traversal, making vector and hybrid searches more efficient under filter conditions. Unlike some pre-existing methods, ACORN is filter-agnostic and does not require advance knowledge of filter criteria, enabling robust and high-performance search even with low correlation between filters and queries.
Evaluating with Lynx...
1.0
True
- The QUESTION asks for the meaning of the term "ACORN".
- The CONTEXT provides information about a new filter strategy called ANN Constraint-Optimized Retrieval Network, which is referred to as ACORN.
- The ANSWER states that ACORN stands for ANN Constraint-Optimized Retrieval Network, which directly match

In [33]:
query = "How does Quantization work?"
run_and_evaluate_query(query)

Quantization is a technique used to compress data, particularly vectors, by reducing the amount of information stored per dimension.

In the context of vectors (commonly used in machine learning and search applications), quantization works by encoding each dimension of a float vector with fewer bits. A common and powerful form is Binary Quantization (BQ), where each dimension is represented by a single bit: 1 if the value is positive and 0 if it is negative. This drastically reduces memory usage—as much as 32x compared to storing each dimension as a 32-bit float—and accelerates computations, as operations on binary vectors (using bitwise operations like XOR) are extremely fast.

Here's how the process essentially works: imagine each vector is like a home address, with each number describing how to reach a location in space. Normally, these details are very precise, but take up a lot of space. Quantization keeps only the sign (direction) for each dimension; with binary quantization, it’

In [34]:
query = "What are Matryoshka embeddings?"
run_and_evaluate_query(query)

Matryoshka embeddings, also known as embeddings trained with Matryoshka Representation Learning (MRL), are a type of vector embedding designed to offer flexible trade-offs between accuracy, storage size, and computational efficiency. Unlike traditional embeddings where all dimensions are equally important, Matryoshka embeddings are structured so that the information is concentrated in the earlier dimensions of the vector. Subsequent dimensions add finer details, much like adding higher resolutions to an image.

This organization allows you to use only a subset of the vector's dimensions—such as the first 8, 16, 32, or any other prefix—to achieve increasingly detailed representations, depending on your accuracy or resource requirements. This enables you to store more embeddings at lower precision or cost and to search faster, with minimal loss in accuracy.

Matryoshka Representation Learning works by modifying the training loss function: the model is trained to perform well not just wit